# 04 - DnD Skill Checks with Live Tool Trace

This notebook stress-tests tool-calling with a manual iterative loop over `LLMAdapter.request_with_tools(...)`.

Goals:
- Expose a random number generator tool for dice rolling.
- Run repeated DnD-style skill checks using `d20 + stat modifier`.
- Push high tool-call volume (minimum gate) with stable completion criteria.
- Print an observable, live trace of assistant summaries plus every tool call/result.

Notes:
- This notebook prints operational decision summaries and execution traces.
- It does **not** attempt to surface hidden internal reasoning.


In [1]:
import json
import random
import re
import sys
import time
from pathlib import Path
from typing import Any

# Ensure repo root is importable when running this notebook from its folder.
repo_root = Path.cwd()
while repo_root != repo_root.parent and not (repo_root / "orchestrator").exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from orchestrator.llm_interaction.adapter import LLMAdapter, LLMError

MODEL = "gpt-oss:20b"
MAX_ITERATIONS = 80
TARGET_CHECKS = 5
MIN_TOOL_CALLS = 30
TRACE_PREVIEW_CHARS = 500

# Maximum-detail tracing controls
FULL_TRACE = True
SHOW_THINKING_TRACE = True
SHOW_RAW_RESPONSE = True
SHOW_RESPONSE_STATS = True
PRINT_FULL_TRACE_AFTER_RUN = True
PRINT_FULL_RESPONSE_SNAPSHOTS_AFTER_RUN = True

adapter = LLMAdapter(model=MODEL, verbose=False)


In [2]:
CHARACTER_SHEET = {
    "name": "Nyx Embervale",
    "class": "Rogue",
    "level": 5,
    "stats": {
        "strength": 10,
        "dexterity": 17,
        "constitution": 13,
        "intelligence": 14,
        "wisdom": 12,
        "charisma": 15,
    },
    "skills": {
        "acrobatics": "dexterity",
        "athletics": "strength",
        "deception": "charisma",
        "insight": "wisdom",
        "investigation": "intelligence",
        "perception": "wisdom",
        "persuasion": "charisma",
        "sleight_of_hand": "dexterity",
        "stealth": "dexterity",
        "survival": "wisdom",
    },
}

RUN_STATE = {
    "check_log": [],
    "total_tool_calls": 0,
    "current_check_id": 1,
    "session_notes": [],
    "last_observed": {},
}

ENCOUNTER_PROMPT = (
    "Run a dangerous infiltration challenge at the Shattered Beacon. "
    "I want at least five distinct skill checks across stealth, investigation, deception, and athletics-like pressure. "
    "Make the scene dynamic, escalating, and concrete. Keep going until the objective is resolved."
)


def reset_run_state() -> None:
    RUN_STATE["check_log"] = []
    RUN_STATE["total_tool_calls"] = 0
    RUN_STATE["current_check_id"] = 1
    RUN_STATE["session_notes"] = []
    RUN_STATE["last_observed"] = {}


def ability_modifier(score: int) -> int:
    return (int(score) - 10) // 2


In [3]:
def read_character_sheet() -> dict[str, Any]:
    return json.loads(json.dumps(CHARACTER_SHEET))


def list_stats() -> dict[str, int]:
    return dict(CHARACTER_SHEET["stats"])


def list_skills() -> dict[str, str]:
    return dict(CHARACTER_SHEET["skills"])


def get_skill_stat(skill: str) -> dict[str, Any]:
    key = skill.strip().lower()
    skills = CHARACTER_SHEET["skills"]
    if key not in skills:
        return {
            "ok": False,
            "error": f"Unknown skill '{skill}'. Available: {', '.join(sorted(skills.keys()))}",
        }
    return {"ok": True, "skill": key, "stat": skills[key]}


def get_stat_modifier(stat: str) -> dict[str, Any]:
    key = stat.strip().lower()
    stats = CHARACTER_SHEET["stats"]
    if key not in stats:
        return {
            "ok": False,
            "error": f"Unknown stat '{stat}'. Available: {', '.join(sorted(stats.keys()))}",
        }
    score = int(stats[key])
    mod = ability_modifier(score)
    return {"ok": True, "stat": key, "score": score, "modifier": mod}


def random_int(low: int, high: int) -> dict[str, Any]:
    if int(low) > int(high):
        raise ValueError("low must be <= high")
    value = random.randint(int(low), int(high))
    return {"low": int(low), "high": int(high), "value": value}


def compute_total(roll: int, modifier: int) -> dict[str, Any]:
    r = int(roll)
    m = int(modifier)
    return {"roll": r, "modifier": m, "total": r + m}


def compare_vs_dc(total: int, dc: int) -> dict[str, Any]:
    t = int(total)
    target = int(dc)
    return {
        "total": t,
        "dc": target,
        "success": t >= target,
        "margin": t - target,
    }


def record_skill_check(
    skill: str,
    stat: str,
    dc: int,
    roll: int,
    modifier: int,
    total: int,
    success: bool,
    note: str = "",
) -> dict[str, Any]:
    skill_key = skill.strip().lower()
    stat_key = stat.strip().lower()

    entry = {
        "id": RUN_STATE["current_check_id"],
        "skill": skill_key,
        "stat": stat_key,
        "dc": int(dc),
        "roll": int(roll),
        "modifier": int(modifier),
        "total": int(total),
        "success": bool(success),
        "note": note.strip(),
    }

    RUN_STATE["check_log"].append(entry)
    RUN_STATE["current_check_id"] += 1

    if entry["note"]:
        RUN_STATE["session_notes"].append(entry["note"])

    return {
        "recorded": entry,
        "completed_checks": len(RUN_STATE["check_log"]),
        "remaining_checks_to_target": max(0, TARGET_CHECKS - len(RUN_STATE["check_log"])),
    }


def get_progress() -> dict[str, Any]:
    checks = RUN_STATE["check_log"]
    last = checks[-1] if checks else None
    return {
        "completed_checks": len(checks),
        "target_checks": TARGET_CHECKS,
        "total_tool_calls": RUN_STATE["total_tool_calls"],
        "min_tool_calls": MIN_TOOL_CALLS,
        "latest_check": last,
    }


In [4]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "read_character_sheet",
            "description": "Read the full player character sheet.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "list_stats",
            "description": "List the six core stats and values.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "list_skills",
            "description": "List available skills and their linked stat.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_skill_stat",
            "description": "Resolve which stat applies to a skill.",
            "parameters": {
                "type": "object",
                "properties": {
                    "skill": {"type": "string"}
                },
                "required": ["skill"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_stat_modifier",
            "description": "Convert a stat score to a modifier.",
            "parameters": {
                "type": "object",
                "properties": {
                    "stat": {"type": "string"}
                },
                "required": ["stat"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "random_int",
            "description": "Generate a random integer in [low, high]. Use for d20 rolls.",
            "parameters": {
                "type": "object",
                "properties": {
                    "low": {"type": "integer", "minimum": 1, "maximum": 100},
                    "high": {"type": "integer", "minimum": 1, "maximum": 100},
                },
                "required": ["low", "high"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "compute_total",
            "description": "Compute roll + modifier total.",
            "parameters": {
                "type": "object",
                "properties": {
                    "roll": {"type": "integer"},
                    "modifier": {"type": "integer"},
                },
                "required": ["roll", "modifier"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "compare_vs_dc",
            "description": "Compare total against DC and return success/margin.",
            "parameters": {
                "type": "object",
                "properties": {
                    "total": {"type": "integer"},
                    "dc": {"type": "integer", "minimum": 5, "maximum": 30},
                },
                "required": ["total", "dc"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "record_skill_check",
            "description": "Persist one resolved skill check in the run log.",
            "parameters": {
                "type": "object",
                "properties": {
                    "skill": {"type": "string"},
                    "stat": {"type": "string"},
                    "dc": {"type": "integer", "minimum": 5, "maximum": 30},
                    "roll": {"type": "integer", "minimum": 1, "maximum": 20},
                    "modifier": {"type": "integer", "minimum": -10, "maximum": 15},
                    "total": {"type": "integer", "minimum": -10, "maximum": 40},
                    "success": {"type": "boolean"},
                    "note": {"type": "string"},
                },
                "required": ["skill", "stat", "dc", "roll", "modifier", "total", "success"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_progress",
            "description": "Read run progress including completed checks and tool-call count.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
]


In [5]:
TOOL_IMPL = {
    "read_character_sheet": read_character_sheet,
    "list_stats": list_stats,
    "list_skills": list_skills,
    "get_skill_stat": get_skill_stat,
    "get_stat_modifier": get_stat_modifier,
    "random_int": random_int,
    "compute_total": compute_total,
    "compare_vs_dc": compare_vs_dc,
    "record_skill_check": record_skill_check,
    "get_progress": get_progress,
}


def ts() -> str:
    return time.strftime("%H:%M:%S")


def preview(text: str, limit: int = TRACE_PREVIEW_CHARS) -> str:
    if text is None:
        return ""
    text = str(text).strip()
    if FULL_TRACE:
        return text
    if len(text) <= limit:
        return text
    return text[:limit] + f" ... [truncated {len(text) - limit} chars]"


def trace_print(tag: str, message: str, trace_lines: list[str] | None = None) -> None:
    line = f"[{ts()}] [{tag}] {message}"
    print(line, flush=True)
    if trace_lines is not None:
        trace_lines.append(line)


def response_to_dict(response: Any) -> dict[str, Any]:
    if hasattr(response, "model_dump"):
        try:
            data = response.model_dump(exclude_none=False)
            if isinstance(data, dict):
                return data
        except Exception:
            pass

    if isinstance(response, dict):
        return response

    return {
        "_type": type(response).__name__,
        "repr": repr(response),
    }


def extract_message_dict(response: Any) -> dict[str, Any]:
    data = response_to_dict(response)
    msg = data.get("message") if isinstance(data, dict) else None
    if isinstance(msg, dict):
        return msg
    if hasattr(msg, "model_dump"):
        try:
            dumped = msg.model_dump(exclude_none=False)
            if isinstance(dumped, dict):
                return dumped
        except Exception:
            pass
    return {}


def extract_thinking_text(response: Any) -> str:
    msg = extract_message_dict(response)
    thinking = msg.get("thinking", "")

    if isinstance(thinking, list):
        return "".join(str(x) for x in thinking)

    return str(thinking or "")


def normalize_tool_calls(response: Any) -> list[dict[str, Any]]:
    out: list[dict[str, Any]] = []
    raw_calls = adapter._extract_tool_calls(response)

    for i, call in enumerate(raw_calls):
        if not isinstance(call, dict):
            continue

        fn = call.get("function", {})
        name = fn.get("name") or call.get("name")
        args = fn.get("arguments", {})

        if isinstance(args, str):
            try:
                args = json.loads(args)
            except json.JSONDecodeError:
                args = {}

        if not isinstance(args, dict):
            args = {}

        if not name:
            continue

        out.append(
            {
                "id": call.get("id") or f"call_{i}",
                "name": str(name),
                "arguments": args,
                "raw": call,
                "source": "structured",
            }
        )

    return out


def extract_fallback_tool_calls_from_text(text: str) -> list[dict[str, Any]]:
    if not text:
        return []

    calls: list[dict[str, Any]] = []

    # Pass 1: line-level JSON objects (common with pseudo tool output)
    for line in text.splitlines():
        stripped = line.strip().rstrip(",")
        if not stripped.startswith("{") or not stripped.endswith("}"):
            continue
        try:
            obj = json.loads(stripped)
        except json.JSONDecodeError:
            continue
        if not isinstance(obj, dict):
            continue
        name = obj.get("name")
        params = obj.get("parameters", {})
        if not name or not isinstance(params, dict):
            continue
        calls.append(
            {
                "id": f"text_line_{len(calls)}",
                "name": str(name),
                "arguments": params,
                "raw": {
                    "function": {
                        "name": str(name),
                        "arguments": params,
                    }
                },
                "source": "text_fallback",
            }
        )

    # Pass 2: multiline regex extraction as backup
    pattern = re.compile(
        r'\{\s*"name"\s*:\s*"(?P<name>[^"]+)"\s*,\s*"parameters"\s*:\s*(?P<params>\{.*?\})\s*\}',
        flags=re.DOTALL,
    )

    for match in pattern.finditer(text):
        name = match.group("name")
        params_raw = match.group("params")
        try:
            params = json.loads(params_raw)
        except json.JSONDecodeError:
            continue
        if not isinstance(params, dict):
            continue

        duplicate = any(c["name"] == str(name) and c["arguments"] == params for c in calls)
        if duplicate:
            continue

        calls.append(
            {
                "id": f"text_regex_{len(calls)}",
                "name": str(name),
                "arguments": params,
                "raw": {
                    "function": {
                        "name": str(name),
                        "arguments": params,
                    }
                },
                "source": "text_fallback",
            }
        )

    return calls


def update_last_observed(name: str, payload: dict[str, Any]) -> None:
    if not payload.get("ok"):
        return

    result = payload.get("result")
    if not isinstance(result, dict):
        return

    observed = RUN_STATE.setdefault("last_observed", {})

    if name == "get_skill_stat":
        if result.get("ok"):
            observed["skill"] = result.get("skill")
            observed["stat"] = result.get("stat")
    elif name == "get_stat_modifier":
        if result.get("ok"):
            observed["stat"] = result.get("stat")
            observed["modifier"] = result.get("modifier")
    elif name == "random_int":
        observed["roll"] = result.get("value")
    elif name == "compute_total":
        observed["roll"] = result.get("roll")
        observed["modifier"] = result.get("modifier")
        observed["total"] = result.get("total")
    elif name == "compare_vs_dc":
        observed["total"] = result.get("total")
        observed["dc"] = result.get("dc")
        observed["success"] = result.get("success")


def hydrate_record_skill_check_args(arguments: dict[str, Any]) -> dict[str, Any]:
    observed = RUN_STATE.get("last_observed", {})
    merged = dict(arguments)

    skill = str(merged.get("skill") or observed.get("skill") or "stealth").strip().lower()
    stat = str(merged.get("stat") or observed.get("stat") or CHARACTER_SHEET["skills"].get(skill, "dexterity")).strip().lower()
    modifier = int(merged.get("modifier", observed.get("modifier", ability_modifier(CHARACTER_SHEET["stats"].get(stat, 10)))))
    roll = int(merged.get("roll", observed.get("roll", 10)))
    total = int(merged.get("total", observed.get("total", roll + modifier)))
    dc = int(merged.get("dc", observed.get("dc", 12)))
    success = bool(merged.get("success", observed.get("success", total >= dc)))
    note = str(merged.get("note", "")).strip()

    return {
        "skill": skill,
        "stat": stat,
        "dc": dc,
        "roll": roll,
        "modifier": modifier,
        "total": total,
        "success": success,
        "note": note,
    }


def execute_tool(name: str, arguments: dict[str, Any], trace_lines: list[str]) -> dict[str, Any]:
    RUN_STATE["total_tool_calls"] += 1
    call_no = RUN_STATE["total_tool_calls"]

    trace_print("TOOL-CALL", f"#{call_no} {name} args={preview(json.dumps(arguments, ensure_ascii=True))}", trace_lines)

    fn = TOOL_IMPL.get(name)
    if fn is None:
        payload = {"ok": False, "error": f"Unknown tool: {name}"}
        trace_print("TOOL-RESULT", f"#{call_no} {name} -> {payload['error']}", trace_lines)
        return payload

    call_args = dict(arguments)
    if name == "record_skill_check":
        call_args = hydrate_record_skill_check_args(call_args)
        trace_print("TOOL-NORMALIZE", f"record_skill_check hydrated args={preview(json.dumps(call_args, ensure_ascii=True))}", trace_lines)

    try:
        result = fn(**call_args)
        payload = {"ok": True, "result": result}
    except Exception as exc:
        payload = {"ok": False, "error": str(exc)}

    update_last_observed(name, payload)

    trace_print("TOOL-RESULT", f"#{call_no} {name} -> {preview(json.dumps(payload, ensure_ascii=True))}", trace_lines)
    return payload


In [6]:
SYSTEM_PROMPT = f"""
You are a high-discipline DnD skill-check engine.

Rules:
1. Never fabricate dice rolls, modifiers, totals, or outcomes. Use tools only.
2. For EACH check, use this exact sequence:
   get_skill_stat -> get_stat_modifier -> random_int -> compute_total -> compare_vs_dc -> record_skill_check
3. Use d20 rolls via random_int(low=1, high=20).
4. Run at least {TARGET_CHECKS} checks before finishing.
5. Total tool calls must reach at least {MIN_TOOL_CALLS} before finishing.
6. Keep checks varied across this challenge context and choose realistic DCs (10-20).
7. Before any tool calls in each assistant turn, output:
   Decision Summary: <operational next-step summary>
8. This runtime logs full reasoning and full raw model output. Provide maximum-detail reasoning in your thinking stream.
9. Use structured tool calling. Do NOT simulate tool calls as plain text JSON.
10. When all conditions are satisfied, output:
    Final Summary: <concise narrative recap and check outcomes>
"""


In [7]:
def run_live_skill_check_loop(user_prompt: str) -> dict[str, Any]:
    trace_lines: list[str] = []
    response_snapshots: list[dict[str, Any]] = []
    start = time.time()

    messages: list[dict[str, Any]] = [
        {
            "role": "user",
            "content": "Character sheet (source of truth):\n" + json.dumps(CHARACTER_SHEET, indent=2),
        },
        {
            "role": "user",
            "content": user_prompt,
        },
    ]

    final_answer = ""

    trace_print("RUN", f"model={MODEL} max_iterations={MAX_ITERATIONS}", trace_lines)
    trace_print("RUN", f"targets: checks>={TARGET_CHECKS}, tool_calls>={MIN_TOOL_CALLS}", trace_lines)

    for iteration in range(1, MAX_ITERATIONS + 1):
        trace_print("ITER", f"{iteration}/{MAX_ITERATIONS} starting", trace_lines)

        try:
            response = adapter.request_with_tools(
                stage="notebook_skill_trace",
                system_prompt=SYSTEM_PROMPT,
                messages=messages,
                tools=TOOLS,
            )
        except LLMError as exc:
            trace_print("MODEL-ERROR", str(exc), trace_lines)
            trace_print("MODEL-ERROR", f"Configured model is '{MODEL}'. Run `ollama list` and adjust MODEL if needed.", trace_lines)
            return {
                "status": "error",
                "error": str(exc),
                "messages": messages,
                "trace_lines": trace_lines,
                "response_snapshots": response_snapshots,
                "progress": get_progress(),
                "check_log": list(RUN_STATE["check_log"]),
                "duration_s": round(time.time() - start, 3),
            }

        response_dict = response_to_dict(response)
        response_snapshots.append(response_dict)

        if SHOW_RESPONSE_STATS:
            stats = {
                "model": response_dict.get("model"),
                "done_reason": response_dict.get("done_reason"),
                "prompt_eval_count": response_dict.get("prompt_eval_count"),
                "eval_count": response_dict.get("eval_count"),
                "total_duration": response_dict.get("total_duration"),
                "prompt_eval_duration": response_dict.get("prompt_eval_duration"),
                "eval_duration": response_dict.get("eval_duration"),
            }
            trace_print("MODEL-STATS", json.dumps(stats, ensure_ascii=True), trace_lines)

        if SHOW_RAW_RESPONSE:
            trace_print("RAW-RESPONSE", json.dumps(response_dict, indent=2, ensure_ascii=True), trace_lines)

        assistant_text = adapter._extract_content(response).strip()
        thinking_text = extract_thinking_text(response).strip()

        if assistant_text:
            trace_print("ASSISTANT-CONTENT", preview(assistant_text), trace_lines)
        else:
            trace_print("ASSISTANT-CONTENT", "<empty>", trace_lines)

        if SHOW_THINKING_TRACE:
            if thinking_text:
                trace_print("ASSISTANT-THINKING", preview(thinking_text), trace_lines)
            else:
                trace_print("ASSISTANT-THINKING", "<none>", trace_lines)

        tool_calls = normalize_tool_calls(response)

        if not tool_calls:
            fallback_source = assistant_text
            if thinking_text:
                fallback_source = f"{assistant_text}\n{thinking_text}" if assistant_text else thinking_text
            fallback_calls = extract_fallback_tool_calls_from_text(fallback_source)
            if fallback_calls:
                tool_calls = fallback_calls
                trace_print(
                    "FALLBACK",
                    f"Recovered {len(tool_calls)} text-encoded tool calls from assistant content/thinking.",
                    trace_lines,
                )

        if tool_calls:
            messages.append(
                {
                    "role": "assistant",
                    "content": assistant_text,
                    "tool_calls": [c["raw"] for c in tool_calls],
                }
            )

            tool_names_this_turn: list[str] = []

            for call in tool_calls:
                tool_names_this_turn.append(call["name"])
                trace_print("TOOL-SOURCE", f"{call['name']} via {call.get('source', 'unknown')}", trace_lines)
                tool_result = execute_tool(call["name"], call["arguments"], trace_lines)
                messages.append(
                    {
                        "role": "tool",
                        "tool_name": call["name"],
                        "content": json.dumps(tool_result, separators=(",", ":"), ensure_ascii=True),
                    }
                )

            if "compare_vs_dc" in tool_names_this_turn and "record_skill_check" not in tool_names_this_turn:
                trace_print(
                    "AUTO-REPAIR",
                    "compare_vs_dc observed without record_skill_check; auto-recording from observed tool state.",
                    trace_lines,
                )
                auto_payload = execute_tool("record_skill_check", {}, trace_lines)
                messages.append(
                    {
                        "role": "tool",
                        "tool_name": "record_skill_check",
                        "content": json.dumps(auto_payload, separators=(",", ":"), ensure_ascii=True),
                    }
                )

            progress = get_progress()
            trace_print(
                "PROGRESS",
                (
                    f"checks={progress['completed_checks']}/{progress['target_checks']} | "
                    f"tool_calls={progress['total_tool_calls']}/{progress['min_tool_calls']}"
                ),
                trace_lines,
            )
            continue

        progress = get_progress()
        checks_ok = progress["completed_checks"] >= TARGET_CHECKS
        calls_ok = progress["total_tool_calls"] >= MIN_TOOL_CALLS

        if checks_ok and calls_ok and assistant_text:
            final_answer = assistant_text
            messages.append({"role": "assistant", "content": assistant_text})
            trace_print("COMPLETE", "exit gates satisfied; loop finished", trace_lines)
            return {
                "status": "completed",
                "final_answer": final_answer,
                "messages": messages,
                "trace_lines": trace_lines,
                "response_snapshots": response_snapshots,
                "progress": progress,
                "check_log": list(RUN_STATE["check_log"]),
                "duration_s": round(time.time() - start, 3),
            }

        blockers = []
        if not checks_ok:
            blockers.append(f"completed_checks={progress['completed_checks']} < {TARGET_CHECKS}")
        if not calls_ok:
            blockers.append(f"total_tool_calls={progress['total_tool_calls']} < {MIN_TOOL_CALLS}")
        if not assistant_text:
            blockers.append("assistant message was empty")

        block_reason = "; ".join(blockers)
        trace_print("STOP-GATE", f"Completion blocked: {block_reason}", trace_lines)

        if assistant_text:
            messages.append({"role": "assistant", "content": assistant_text})

        messages.append(
            {
                "role": "user",
                "content": (
                    "Completion blocked: minimum checks/calls not met. "
                    "Continue with more tool calls. Use structured tool-calling (not plain-text tool JSON). "
                    "For each new check use exact sequence "
                    "get_skill_stat -> get_stat_modifier -> random_int -> compute_total -> "
                    "compare_vs_dc -> record_skill_check."
                ),
            }
        )

    trace_print("HARD-STOP", f"Reached MAX_ITERATIONS={MAX_ITERATIONS}", trace_lines)
    return {
        "status": "max_iterations",
        "final_answer": "Stopped due to max iterations.",
        "messages": messages,
        "trace_lines": trace_lines,
        "response_snapshots": response_snapshots,
        "progress": get_progress(),
        "check_log": list(RUN_STATE["check_log"]),
        "duration_s": round(time.time() - start, 3),
    }


In [8]:
reset_run_state()

result = run_live_skill_check_loop(ENCOUNTER_PROMPT)

print("\n===== RUN RESULT =====")
print("status:", result.get("status"))
print("duration_s:", result.get("duration_s"))

progress = result.get("progress", {})
print("completed_checks:", progress.get("completed_checks"))
print("total_tool_calls:", progress.get("total_tool_calls"))

check_log = result.get("check_log", [])
successes = sum(1 for c in check_log if c.get("success"))
rate = (successes / len(check_log) * 100.0) if check_log else 0.0

print("\n===== CHECK SUMMARY =====")
print(f"checks: {len(check_log)}")
print(f"successes: {successes}")
print(f"success_rate: {rate:.1f}%")

if check_log:
    print("\nID | Skill | Stat | DC | Roll | Mod | Total | Success")
    print("-" * 72)
    for c in check_log:
        print(
            f"{c['id']:>2} | {c['skill']:<14} | {c['stat']:<12} | "
            f"{c['dc']:>2} | {c['roll']:>2} | {c['modifier']:>3} | {c['total']:>3} | {str(c['success'])}"
        )

print("\n===== FINAL ASSISTANT SUMMARY =====")
print(result.get("final_answer", "<none>"))

trace_lines = result.get("trace_lines", [])
print("\n===== TRACE LINE COUNT =====")
print(len(trace_lines))

if PRINT_FULL_TRACE_AFTER_RUN:
    print("\n===== FULL TRACE LINES =====")
    for line in trace_lines:
        print(line)
else:
    print("\n===== TRACE LINES (FIRST 12) =====")
    for line in trace_lines[:12]:
        print(line)

    print("\n===== TRACE LINES (LAST 12) =====")
    for line in trace_lines[-12:]:
        print(line)

snapshots = result.get("response_snapshots", [])
print("\n===== RESPONSE SNAPSHOT COUNT =====")
print(len(snapshots))

if PRINT_FULL_RESPONSE_SNAPSHOTS_AFTER_RUN:
    print("\n===== FULL RESPONSE SNAPSHOTS =====")
    for i, snap in enumerate(snapshots, start=1):
        print(f"\n--- SNAPSHOT {i} ---")
        print(json.dumps(snap, indent=2, ensure_ascii=True))


[21:34:21] [RUN] model=gpt-oss:20b max_iterations=80
[21:34:21] [RUN] targets: checks>=5, tool_calls>=30
[21:34:21] [ITER] 1/80 starting
[21:37:59] [MODEL-STATS] {"model": "gpt-oss:20b", "done_reason": "stop", "prompt_eval_count": 828, "eval_count": 6898, "total_duration": 218229134875, "prompt_eval_duration": 1038750125, "eval_duration": 132574647004}
[21:37:59] [RAW-RESPONSE] {
  "model": "gpt-oss:20b",
  "created_at": "2026-02-20T02:37:59.651055Z",
  "done": true,
  "done_reason": "stop",
  "total_duration": 218229134875,
  "load_duration": 101601750,
  "prompt_eval_count": 828,
  "prompt_eval_duration": 1038750125,
  "eval_count": 6898,
  "eval_duration": 132574647004,
  "message": {
    "role": "assistant",
    "content": "",
    "thinking": "We need to produce a dynamic infiltration scenario. We must perform at least 5 skill checks. Must use the exact sequence of tool calls per check. Each check must be distinct across stealth, investigation, deception, athletics-like. We must us